In [1]:
import numpy as np
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dp = np.loadtxt('../data/GM/2gaussian.txt')

In [3]:
P = lambda X, mu, s: np.linalg.det(s) ** -.5 ** (2 * np.pi) ** (-X.shape[1]/2.) \
                * np.exp(-.5 * np.einsum('ij, ij -> i',\
                        X - mu, np.dot(np.linalg.inv(s) , (X - mu).T).T ) )

In [27]:
def init(X, k):
    
    n_rows, dims = X.shape
    
    mu = X[np.random.choice(n_rows, 2, False)]
    sigma = [np.eye(k) for _ in range(k)]
    w = [1/k for _ in range(k)]
    pi = np.zeros(shape=(n_rows, k))
    
    return mu, sigma, w, pi

In [132]:
def e_step(X, k, mu, sigma, w, pi):
    
    n_rows, dims = X.shape
                        
    for _k in range(k):
        pi[:, _k] = w[_k] * P(dp, mu[_k], sigma[_k])
    
    loss = np.sum(np.log(pi.sum(axis=1)))
    pi = pi.T / pi.sum(axis=1)
    
    return pi.T, loss

In [133]:
def m_step(X, k, pi, mu, sigma, w):
    
    N_k = pi.sum(axis=0)
                
    for _k in range(k):
        
        mu[_k] = 1/N_k[_k] * (pi[:, _k] * X.T).sum(axis=1)
        
        x_mu = X - mu[_k]
        sigma[_k] = np.array(1/N_k[_k] * np.dot(np.multiply(x_mu.T, pi[:, _k]), x_mu))
        
        w[_k] = N_k[_k] / X.shape[0]
                                        
    return mu, sigma, w, N_k

In [137]:
def do_em(X, k, max_epochs=100):
    
    np.random.seed(666)
    mu, sigma, weights, pi = init(X, k)
    
    p_loss = 0
    
    for epoch in range(max_epochs):
                
        # e step
        pi, loss = e_step(X, k, mu, sigma, weights, pi)
        
        # m step
        mu, sigma, coeff, N_k = m_step(X, k, pi, mu, sigma, weights)
        
        # check convergence
        if np.abs(p_loss - loss) < 0.0001:
            print('Convergenve at epoch {0}'.format(epoch))
            break
        p_loss = loss
        
    print('Final loss: {0}'.format(loss))
    print(N_k)

In [138]:
do_em(dp, 2, 1000)

Convergenve at epoch 42
Final loss: -10445.603358566954
[ 4095.91852725  1904.08147275]


In [139]:
print(mu)

[[ 5.72617704  3.44160626]
 [ 5.59702385  3.94831234]]


In [140]:
print(sigma)

[array([[ 4.36201587,  1.44636342],
       [ 1.44636342,  1.87279356]]), array([[ 4.84214949,  0.88021396],
       [ 0.88021396,  1.66588995]])]
